In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [107]:
df = pd.read_csv("./DatingAppReviewsDataset.csv", index_col=0)
print(df.isna().any())

print("Name et Review contiennent des NaNs on va donc les nettoyer")

Name          True
Review        True
Rating       False
#ThumbsUp    False
Date&Time    False
App          False
dtype: bool
Name et Review contiennent des NaNs on va donc les nettoyer


In [109]:
#On supprime les rows si Review, Rating ou App sont des nan. Des lignes avec #ThumbsUp,	Date&Time et Name étant des NAN existent 
# mais leur impact est limité dans notre analyse 

df.dropna(inplace=True, subset=["Review", "Rating", "App"])
df["Name"].fillna("Not available", inplace=True)

print(df.isna().any())
print("On n'a plus aucun NaNs et on s'est débarrasé des rows ayant des reviews vides.")

Name         False
Review       False
Rating       False
#ThumbsUp    False
Date&Time    False
App          False
dtype: bool
On n'a plus aucun NaNs et on s'est débarrasé des rows ayant des reviews vides.
